In [16]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [10]:
# Step 2: Load the dataset
data = pd.read_csv('dataset/heart.csv')  # Adjust the file path if necessary

In [11]:
# Step 3: Data Preprocessing

# Check for missing values in the dataset
print(data.isnull().sum())

# In this dataset, there are no missing values. If there were missing values, you could fill or drop them.
# You can also check for duplicate rows
print(data.duplicated().sum())

# Feature columns (X) and target column (y)
X = data.drop('target', axis=1)  # Features
y = data['target']  # Target variable (1: heart disease, 0: no heart disease)

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64
723


In [12]:
# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Step 5: Standardize the data (Scaling)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform the training data
X_test = scaler.transform(X_test)  # Only transform the testing data

In [14]:
# Initialize and fit the scaler
scaler = StandardScaler()
scaler.fit(X_train)

# Save the fitted scaler
joblib.dump(scaler, 'scaler.pkl')  

print("Scaler saved successfully!")

Scaler saved successfully!


In [17]:
# Step 6: Model Selection - Random Forest, SVM, KNN
models = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

In [18]:
# Step 7: Train and evaluate each model
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    print(f"\n{model_name} Results:")
    print("Accuracy Score:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    
    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=5)
    print("Cross-validation Scores:", cv_scores)
    print("Average Cross-validation Score:", np.mean(cv_scores))


Training Random Forest...

Random Forest Results:
Accuracy Score: 0.9853658536585366
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205

Confusion Matrix:
 [[102   0]
 [  3 100]]
Cross-validation Scores: [1.         1.         1.         1.         0.98536585]
Average Cross-validation Score: 0.9970731707317073

Training SVM...

SVM Results:
Accuracy Score: 0.8878048780487805
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.83      0.88       102
           1       0.85      0.94      0.89       103

    accuracy                           0.89       205
   macro avg       0.89      0.89      0.89       205
weighted avg       0.89   

In [19]:
# Step 8: Model Selection

# Hyperparameter Tuning for Random Forest (using GridSearchCV)

# Define the model
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Accuracy: {grid_search.best_score_}")

# Get the best model
best_rf_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_best = best_rf_model.predict(X_test)
print("Best Random Forest Accuracy:", accuracy_score(y_test, y_pred_best))
print("Classification Report:\n", classification_report(y_test, y_pred_best))



Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'bootstrap': False, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Best Cross-validation Accuracy: 0.9853658536585366
Best Random Forest Accuracy: 0.9853658536585366
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205



In [ ]:
# Feature Engineering
# a. Interaction Features

# Add interaction features (example: age * cholesterol)
X['age_chol_interaction'] = X['age'] * X['chol']

# Re-split the data after adding new features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the new data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# b. Removing Irrelevant Features

# Example: Drop 'restecg' and 'ca' if they don't contribute much
X = X.drop(columns=['restecg', 'ca'])

# Re-split the data after removing features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the new data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# Assuming 'best_rf_model' is your trained Random Forest model from hyperparameter tuning or training.
# Save the trained model to a file
joblib.dump(best_rf_model, 'heart_disease_rf_model.pkl')

print("Model saved successfully!")

Model saved successfully!
